In [ ]:
import os
import glob
from tqdm import tqdm
from random import randint

import numpy as np
import pydicom

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as anim
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec


import imageio
from skimage.transform import resize
from skimage.util import montage

from IPython.display import Image as show_gif

import warnings
warnings.simplefilter("ignore")

In [ ]:
sample_id = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000'
path_x = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T2w/*.dcm'
start = len('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T2w/Image-')
end = len('.dcm')
path_to_slices = sorted(glob.glob(path_x), key= lambda x: int(x[start:-end]))


The easiest way to render a 3d image is to draw its slices one by one in 2d , for easy viewing we can make a GIF from each slice

In [ ]:
class ImageToGIF:
    """Create GIF without saving image files."""
    def __init__(self,
                 size=(500, 500), 
                 xy_text=(80, 30),
                 dpi=100, 
                 cmap='CMRmap'):

        self.fig = plt.figure()
        self.fig.set_size_inches(size[0] / dpi, size[1] / dpi)
        self.xy_text = xy_text
        self.cmap = cmap
        
        self.ax = self.fig.add_axes([0, 0, 1, 1])
        self.ax.set_xticks([])
        self.ax.set_yticks([])
        self.images = []
 
    def add(self, image, label, with_mask=False):
        plt.set_cmap(self.cmap)
        plt_img = self.ax.imshow(image, animated=True)
        plt_text = self.ax.text(*self.xy_text, label, color='red')
        to_plot = [plt_img, plt_text]
        self.images.append(to_plot)
        plt.close()
 
    def save(self, filename, fps):
        animation = anim.ArtistAnimation(self.fig, self.images)
        animation.save(filename, writer='imagemagick', fps=fps)
        

sample_data_gif = ImageToGIF()
label = sample_id.replace('/', '.').split('.')[-2]
filename = f'{label}_3d_2d.gif'

for i in range(len(path_to_slices)):
    image = pydicom.read_file(path_to_slices[i]).pixel_array
    #mask = np.clip(np.rot90(sample_mask[i]), 0, 1)
    sample_data_gif.add(image, label=f'{label}_{str(i)}')

    
sample_data_gif.save(filename, fps=15)
show_gif(filename, format='png')

## From pixels to voxels

In [ ]:
class Image3dToGIF3d:
    """
    Displaying 3D images in 3d axes.
    Parameters:
        img_dim: shape of cube for resizing.
        figsize: figure size for plotting in inches.
    Step by step explanation - https://terbium.io/2017/12/matplotlib-3d/
    """
    def __init__(self, 
                 img_dim: tuple = (55, 55, 55),
                 figsize: tuple = (15, 10),
                 binary: bool = False,
                 normalizing: bool = True,
                ):
        """Initialization."""
        self.img_dim = img_dim
        print(img_dim)
        self.figsize = figsize
        self.binary = binary
        self.normalizing = normalizing

    def _explode(self, data: np.ndarray):
        """
        Takes: array and return an array twice as large in each dimension,
        with an extra space between each voxel.
        """
        shape_arr = np.array(data.shape)
        size = shape_arr[:3] * 2 - 1
        exploded = np.zeros(np.concatenate([size, shape_arr[3:]]),
                            dtype=data.dtype)
        exploded[::2, ::2, ::2] = data
        return exploded

    def _expand_coordinates(self, indices: np.ndarray):
        """ 
        Parameters:
            indices: coordinats of array with only original values
            (before explode transformaion)
        
        Returns:
        The arrays of values each dimensions (x y z) as arguments needed 
        for the plt.figure.voxels functionto extend coordinates
        for the rendering only colored voxels"""
        x, y, z = indices
        x[1::2, :, :] += 1
        y[:, 1::2, :] += 1
        z[:, :, 1::2] += 1
        return x, y, z
    
    def _normalize(self, arr: np.ndarray):
        """Normilize image value between 0 and 1."""
        arr_min = np.min(arr)
        return (arr - arr_min) / (np.max(arr) - arr_min)

    
    def _scale_by(self, arr: np.ndarray, factor: int = 2):
        """
        Scale 3d Image to factor (guesstimated transformation).
        Parameters:
            arr: 3d image for scalling.
            factor: factor for scalling.
        """
        mean = np.mean(arr)
        return (arr - mean) * factor + mean
    
    def get_transformed_data(self, data: np.ndarray):
        """Data transformation: normalization, scaling, resizing."""
        if self.binary:
            resized_data = resize(data, self.img_dim, preserve_range=True)
            return np.clip(resized_data.astype(np.uint8), 0, 1).astype(np.float32)
            
        norm_data = np.clip(self._normalize(data)-0.1, 0, 1) ** 0.4
        scaled_data = np.clip(self._scale_by(norm_data) - 0.1, 0, 1)
        resized_data = resize(scaled_data, self.img_dim, preserve_range=True)
        
        return resized_data
    
    def plot_cube(self,
                  cube,
                  title: str = '', 
                  init_angle: int = 0,
                  make_gif: bool = False,
                  path_to_save: str = 'filename.gif'
                 ):
        """
        Plot 3d data.
        -> Take array 
        -> return an array twice as large in each dimension,
        (with an extra space between each voxel)
        -> expand coordinates of each voxel for core rendering (to remove gaps)
        because additional fake voxels have been added
        -> set each voxel’s transparency equal to its value.       
        Parameters:
            cube: 3d data
            title: title for figure.
            init_angle: angle for image plot (from 0-360).
            make_gif: if True create gif from every 5th frames from 3d image plot.
            path_to_save: path to save GIF file.
            """

 
        if self.normalizing:
            cube = self._normalize(cube)
            
        facecolors = cm.gist_stern(cube)          
        facecolors[:,:,:,-1] = cube
        facecolors = self._explode(facecolors)

        filled = facecolors[:,:,:,-1] != 0
        x, y, z = self._expand_coordinates(np.indices(np.array(filled.shape) + 1))

        with plt.style.context("dark_background"):

            fig = plt.figure(figsize=self.figsize)
            ax = fig.gca(projection='3d')

            ax.view_init(30, init_angle)
            ax.set_xlim(right = self.img_dim[0] * 2)
            ax.set_ylim(top = self.img_dim[1] * 2)
            ax.set_zlim(top = self.img_dim[2] * 2)
            ax.set_title(title, fontsize=18, y=1.05)

            ax.voxels(x, y, z, filled, facecolors=facecolors, shade=False)

            if make_gif:
                images = []
                for angle in tqdm(range(0, 360, 5)):
                    ax.view_init(30, angle)
                    fname = str(angle) + '.png'

                    plt.savefig(fname, dpi=120, format='png', bbox_inches='tight')
                    images.append(imageio.imread(fname))
                    #os.remove(fname)
                imageio.mimsave(path_to_save, images)
                plt.close()

            else:
                plt.show()

In [ ]:
tensor = np.zeros((512, 512, len(path_to_slices)))
for i in range(len(path_to_slices)):
    image = pydicom.read_file(path_to_slices[i]).pixel_array
    tensor[:,:,i] = image

In [ ]:
title = sample_id.replace(".", "/").split("/")[-1]
filename = title+"_3d.gif"

data_to_3dgif = Image3dToGIF3d(img_dim = (120, 120, 78))
transformed_data = data_to_3dgif.get_transformed_data(np.moveaxis(np.flipud(tensor), [0, 1, 2], [-1, -2, -3]))
data_to_3dgif.plot_cube(
    transformed_data[:77, :100, :55],
    title=title,
    make_gif=True,
    path_to_save=filename
)

show_gif(filename, format='png')

We can also compare images
with T1-weighted, T1CE-weighted, T2-weighted and FLAIR-weighted, and notice that their range of values is different

In [ ]:
NUM = '00012'

path_flair = f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{NUM}/FLAIR/*.dcm'
start_flair = len(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{NUM}/FLAIR/Image-')
end_flair = len('.dcm')
path_to_slices_flair = sorted(glob.glob(path_flair), key= lambda x: int(x[start_flair:-end_flair]))

path_t1w = f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{NUM}/T1w/*.dcm'
start_t1w = len(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{NUM}/T1w/Image-')
end_t1w = len('.dcm')
path_to_slices_t1w = sorted(glob.glob(path_t1w), key= lambda x: int(x[start_t1w:-end_t1w]))

path_t1wce = f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{NUM}/T1wCE/*.dcm'
start_t1wce = len(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{NUM}/T1wCE/Image-')
end_t1wce = len('.dcm')
path_to_slices_t1wce = sorted(glob.glob(path_t1wce), key= lambda x: int(x[start_t1wce:-end_t1wce]))

path_t2w = f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{NUM}/T2w/*.dcm'
start_t2w = len(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{NUM}/T2w/Image-')
end_t2w = len('.dcm')
path_to_slices_t2w = sorted(glob.glob(path_t2w), key= lambda x: int(x[start_t2w:-end_t2w]))


def slices_dcm_to_tensor(pth: str) -> np.ndarray:
    tensor = np.zeros((512, 512,len(pth)))
    for i in range(len(pth)):
        image = pydicom.read_file(pth[i]).pixel_array
        tensor[:,:,i] = image
    return tensor

In [ ]:
flair_arr = slices_dcm_to_tensor(path_to_slices_flair)
t1w_arr = slices_dcm_to_tensor(path_to_slices_t1w)
t1wce_arr = slices_dcm_to_tensor(path_to_slices_t1wce)
t2w_arr = slices_dcm_to_tensor(path_to_slices_t2w)

print(flair_arr.shape, t1w_arr.shape, t1wce_arr.shape, t2w_arr.shape)

If we look at the color scales for the random slices, we can see how many points reach the high or low values (greater than, say, 4000, or less than 500).

In [ ]:
fig = plt.figure(figsize=(20, 10))

gs = gridspec.GridSpec(nrows=2, ncols=4, height_ratios=[1, 1.5])

#  Varying density along a streamline
ax0 = fig.add_subplot(gs[0, 0])
flair = ax0.imshow(flair_arr[:,:,150], cmap='bone')
ax0.set_title("FLAIR", fontsize=18, weight='bold', y=-0.2)
fig.colorbar(flair)

#  Varying density along a streamline
ax1 = fig.add_subplot(gs[0, 1])
t1 = ax1.imshow(t1w_arr[:,:,15], cmap='bone')
ax1.set_title("T1", fontsize=18, weight='bold', y=-0.2)
fig.colorbar(t1)

#  Varying density along a streamline
ax2 = fig.add_subplot(gs[0, 2])
t1ce = ax2.imshow(t1wce_arr[:,:,160], cmap='bone')
ax2.set_title("T1 contrast", fontsize=18, weight='bold', y=-0.2)
fig.colorbar(t1ce)

#  Varying density along a streamline
ax3 = fig.add_subplot(gs[0, 3])
t2 = ax3.imshow(t2w_arr[:,:,175], cmap='bone')
ax3.set_title("T2", fontsize=18, weight='bold', y=-0.2)
fig.colorbar(t2)

plt.show();